## Deep Learning for Computer Vision

### Logo Detection
### Bocconi University


Please, use a GPU session.

In [ ]:
"""
  IMPORTING LIBRARIES
"""

import os
import shutil
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile

### 1. Split the data
We start by importing the dataset from dropbox and splitting it into train, validation and test set. 

In [ ]:
"""
  IMPORTING TAR DATASET FROM DROPBOX
"""

!wget https://www.dropbox.com/s/nkoxs4boe8m48xf/DLCV_logo_project.tar.gz

--2021-12-02 16:18:33--  https://www.dropbox.com/s/nkoxs4boe8m48xf/DLCV_logo_project.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nkoxs4boe8m48xf/DLCV_logo_project.tar.gz [following]
--2021-12-02 16:18:33--  https://www.dropbox.com/s/raw/nkoxs4boe8m48xf/DLCV_logo_project.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd19277bf59dba98a7d654c8c2d.dl.dropboxusercontent.com/cd/0/inline/BbHKlQB-F-355BZgu_vl0MFvB7ef92wuM9xcvp6KcuMyI6xyHt7iUsxmm-cVSnxcQFd3fq2sbyC3z_WWut1wW8bmlaui65JWaTCkhOCf_dJ-DcHk1tG9juLMbjGn3FQCYEgYo-O0XHWPR47eGWf2AWdj/file# [following]
--2021-12-02 16:18:33--  https://ucd19277bf59dba98a7d654c8c2d.dl.dropboxusercontent.com/cd/0/inline/BbHKlQB-F-355BZgu_vl0MFvB7ef92wuM9xcvp6KcuMyI6xyHt7iUs

In [ ]:
!tar -xf DLCV_logo_project.tar.gz # untar dataset

In [ ]:
"""
  CREATING LOCAL PATHS
"""

path = '/content'

# Defining directories' path
data_dir = join(path, 'data')
train_dir = join(data_dir, 'train')
valid_dir = join(data_dir, 'valid')
test_dir = join(data_dir, 'test')

# Creating directories
os.mkdir(data_dir)
os.mkdir(train_dir)
os.mkdir(valid_dir)
os.mkdir(test_dir)

In [ ]:
"""
  FUNCTION THAT RECEIVES THE ORIGINAL PATH OF THE PHOTO FOLDER, 
  THE TARGET PATH FOR THE COPY FOLDER,
  DATAFRAME THAT THE FIRST COLUMN IS THE NAME OF THE PHOTO FILE IN ORIGINAL PATH
"""
def change_directory(original,target,df_photo_filename):
    for index, rows in (df_photo_filename.iterrows()):
        new_original = join(original, rows.values[0])
        new_target = join(target, rows.values[0])
        shutil.copyfile(new_original, new_target)


"""
  SPLITTING FILES USING ALL ORIGINAL PHOTOS AND PANDAS DATAFRAME SAMPLE FOR:
    - 80% TRAIN SET
    - 10% VALID SET
    - 10% TEST SET
"""


# Original data path
original_train = r'/content/DLCV_logo_project/train'

# Check the name of all images in original train to avoid errors
onlyfiles = [f for f in listdir(original_train) if isfile(join(original_train, f))]

# Annotation to dataframe
df = pd.read_csv('/content/DLCV_logo_project/annot_train.csv')
df = df[df['photo_filename'].isin(onlyfiles)] # filter for onlyfiles

# 14 Logos
logos = ["Adidas","Apple Inc.","Chanel","Coca-Cola","Hard Rock Cafe","Mercedes-Benz",
         "NFL","Nike","Pepsi","Puma","Starbucks","The North Face","Toyota","Under Armour"]
df = df[df['class'].isin(logos)]

# Split train dataframe
df_train = df.sample(frac=0.8, random_state=0).reset_index(drop=True)
df_test_valid = df[~df['photo_filename'].isin(df_train['photo_filename'])].reset_index(drop=True)

# Split valid and test dataframes
df_valid = df_test_valid.sample(frac=0.5, random_state=0).reset_index(drop=True)
df_test = df_test_valid[~df_test_valid['photo_filename'].isin(df_valid['photo_filename'])].reset_index(drop=True)

# Final split
df_photo_filename_train = df_train[['photo_filename']]
df_photo_filename_valid = df_valid[['photo_filename']]
df_photo_filename_test = df_test[['photo_filename']]

# Copy splitted files to new origin
change_directory(original_train, train_dir, df_photo_filename_train)
change_directory(original_train, valid_dir, df_photo_filename_valid)
change_directory(original_train, test_dir, df_photo_filename_test)

# Save data frames
df_train.to_csv(join(data_dir, 'train.csv'), index=False)
df_valid.to_csv(join(data_dir, 'valid.csv'), index=False)
df_test.to_csv(join(data_dir, 'test.csv'), index=False)

In [ ]:
"""
  STATISTICS OF THE TRAIN/VALID/TEST SETS
"""

stats_split = dict()

for classe in df['class'].unique():
  total = len(df[df['class']==classe])
  stats = [total,
           len(df_train[df_train['class']==classe])*100/total,
            len(df_valid[df_valid['class']==classe])*100/total,
            len(df_test[df_test['class']==classe])*100/total]
  stats_split[classe] = stats
  
stats_split = pd.DataFrame(stats_split, 
                           columns = stats_split.keys(), 
                           index = ['Original','Train %', 'Valid %', 'Test %']).T
stats_split = stats_split.sort_values(by=['Original'],ascending=False)

stats_split

,Original,Train %,Valid %,Test %
Nike,9566.0,79.960276,9.983274,10.056450
Adidas,8119.0,79.874369,9.902697,10.222934
Starbucks,3660.0,79.699454,10.273224,10.027322
Mercedes-Benz,2089.0,80.947822,8.903782,10.148396
NFL,2079.0,80.952381,10.582011,8.465608
Apple Inc.,1859.0,80.527165,9.252286,10.220549
Under Armour,1467.0,80.027267,10.633947,9.338787
Coca-Cola,1131.0,80.017683,9.549072,10.433245
Puma,964.0,78.734440,10.269710,10.995851
Hard Rock Cafe,954.0,79.559748,10.901468,9.538784


In [ ]:
"""
  REMOVE TAR.GZ FILE TO SAVE LOCAL SPACE
"""
!rm /content/DLCV_logo_project.tar.gz

###2. Create tf records

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3137, done.
remote: Counting objects: 100% (3137/3137), done.
remote: Compressing objects: 100% (2652/2652), done.
remote: Total 3137 (delta 809), reused 1343 (delta 441), pack-reused 0
Receiving objects: 100% (3137/3137), 33.35 MiB | 14.36 MiB/s, done.
Resolving deltas: 100% (809/809), done.


In [ ]:
%%shell
python -m pip install --upgrade pip
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 9.8 MB 8.9 MB/s            
     |████████████████████████████████| 352 kB 41.0 MB/s            
     |████████████████████████████████| 1.8 MB 31.3 MB/s            
     |████████████████████████████████| 22.7 MB 1.8 MB/s             
     |████████████████████████████████| 1.3 MB 34.5 MB/s            
     |████████████████████████████████| 43 kB 1.8 MB/s             
  Preparing metadata (setup.py) ... done


In [ ]:
'''
  FUNCTION TO CONVERT THE IMAGES + ANNOTATIONS
  INTO TF RECORDS USING THE OFFICIAL TUTORIAL
  https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md
'''

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow.compat.v1 as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

"""
  FUNCTION MAPPING NAME TO NUMBER
  NECESSARY TO CREATE THE TF RECORDS FOR THIS SPECIFIC PROJECT
"""
def class_text_to_int(row_label):
    if row_label == 'Nike':
        return 1
    elif row_label == 'Adidas':
        return 2
    elif row_label == 'Starbucks':
        return 3
    elif row_label == 'Apple Inc.':
        return 4
    elif row_label == 'NFL':
        return 5
    elif row_label == 'Mercedes-Benz':
        return 6
    elif row_label == 'Under Armour':
        return 7
    elif row_label == 'Coca-Cola':
        return 8
    elif row_label == 'Hard Rock Cafe':
        return 9
    elif row_label == 'Puma':
        return 10
    elif row_label == 'The North Face':
        return 11
    elif row_label == 'Toyota':
        return 12
    elif row_label == 'Chanel':
        return 13
    elif row_label == 'Pepsi':
        return 14
    else:
        None

"""
  FUNCTION TO GROUP ANNOTATIONS PER IMAGE,
  IN CASE SOME IMAGES HAS MORE THAN ONE ANNOTATION
  NEED THE DATAFRAME WITH ANNOTATIONS AND THE NAME OF THE COLUMN
  WHERE THERE IS THE PHOTO FILE NAME
  DF = DATAFRAME WITH ANNOTATIONS
  GROUP = NAME OF THE COLUMN CONTAINING THE IMAGE NAME
"""
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


"""
  FUNCTION TO CREATE THE TF RECORDS PER IMAGE
  GROUP = SINGLE IMAGE GROUPPED
  PATH = PATH TO IMAGE DIRECTORY
"""
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read() # READ THE IMAGE
    encoded_jpg_io = io.BytesIO(encoded_jpg) # CONVERT TO BYTES
    image = Image.open(encoded_jpg_io) # OPEN IMAGE WITH IMAGE LIBRARY
    width, height = image.size # GET THE WIDHT AND HEIGHT

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


"""
  FUCTION THAT USES THE 2 PREVIOUS FUNCTIONS
  OUTPUT_PATH = PATH TO SAVE THE TF RECORDS
  IMG_DIR = PATH TO IMAGE DIRECTORY
  CSV_INPUT = ANNOTATIONS OF ALL IMAGES FROM IMG_DIR
"""
def generate_tfrecord(output_path,img_dir,csv_input):
    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(img_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'photo_filename')
    for group in grouped: #for each image in groupped images
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print('Successfully created the TFRecords: {}'.format(output_path))

In [ ]:
# CREATING TF RECORDS FOR THE TRAINING IMAGES

output_path = '/content/data/train.tfrecord'
img_dir = '/content/data/train'
csv_input= '/content/data/train.csv'
generate_tfrecord(output_path,img_dir,csv_input) 

Successfully created the TFRecords: /content/data/train.tfrecord


In [ ]:
# CREATING TF RECORDS FOR THE VALIDATION IMAGES

output_path = '/content/data/valid.tfrecord'
img_dir = '/content/data/valid'
csv_input= '/content/data/valid.csv'
generate_tfrecord(output_path,img_dir,csv_input) 

Successfully created the TFRecords: /content/data/valid.tfrecord


The following files were coppied to our shared drive folder:    
- /content/data/test
- /content/data/test.csv
- /content/data/train.csv
- /content/data/valid.csv
- /content/data/train.tfrecord
- /content/data/valid.tfrecord

The others folders/files are not used neither for training nor for testing the model.